In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
%matplotlib inline


/home/riktim/anaconda3/envs/jde/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/riktim/anaconda3/envs/jde/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
pat=pd.read_csv('string.csv',names = ["pattern"])
pat.head(2)
#print(len(df["pattern"][0]))


,pattern
0,'CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHCCCCC...
1,'CCCHHHHHHHHHHHHHHHHHHCHHHHHHHHHHHHHHHHHCHHHHH...


In [3]:
class_file_name=pd.read_csv('id_label.csv',names = ["class", "file_name"])
class_file_name.head(2)

,class,file_name
0,1,'1ab3A.secstr'
1,1,'1abvA.secstr'


In [4]:
pat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 1 columns):
pattern    551 non-null object
dtypes: object(1)
memory usage: 4.4+ KB


In [5]:
pat.describe()

,pattern
count,551
unique,551
top,'CEEEEEECCCCEEEEEEECCCCCCEEEEEEECCCEEEEEECCCCE...
freq,1


In [6]:
data=pd.DataFrame()
data['pattern']=pat['pattern']
data['class_no']=class_file_name['class']


In [7]:
data.tail()

,pattern,class_no
546,'CCCCEEEEEEEECCCCCCCCCCEEEEEECCCCCEEECCCHHHHHH...,4
547,'CCCCCCCCCCCCCCCCEEEEEECHHHCCCEECCCCCCCCCCCCCE...,4
548,'CEEEEEECCHHHHHHHCCCCCEEEEECCCCCCCCEEEEEEECCCC...,4
549,'CCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHCCEEEEEEE...,4
550,'CCCHHHHHHHHHCCCCCCCCCCCHHHHHHHHHHCCCCCCCCCEEE...,4


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 2 columns):
pattern     551 non-null object
class_no    551 non-null int64
dtypes: int64(1), object(1)
memory usage: 8.7+ KB


In [9]:
X = data.pattern
Y = data.class_no
#le = LabelEncoder()
#Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)

In [10]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [11]:
dummy_y.shape

(551, 4)

In [12]:
dummy_y[200]

array([0., 1., 0., 0.], dtype=float32)

In [13]:
X_train,X_test,Y_train,Y_test = train_test_split(X,dummy_y,test_size=0.15)

In [14]:
max_len=int(data['pattern'].str.len().max())
print(max_len)
min_len=int(data['pattern'].str.len().min())
print(min_len)

871
33


In [15]:
max_words = 4
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [16]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(4,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [17]:
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 871)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 871, 50)           200       
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 4)                 1028      
__________

In [18]:
model.fit(sequences_matrix,Y_train,batch_size=1,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])


Train on 374 samples, validate on 94 samples
Epoch 1/10
374/374 [==============================] - 306s 818ms/step - loss: 1.3900 - acc: 0.2219 - val_loss: 1.3872 - val_acc: 0.2660
Epoch 2/10
374/374 [==============================] - 304s 812ms/step - loss: 1.3899 - acc: 0.2513 - val_loss: 1.3862 - val_acc: 0.2660
Epoch 3/10
374/374 [==============================] - 303s 809ms/step - loss: 1.3889 - acc: 0.2299 - val_loss: 1.3841 - val_acc: 0.2872
Epoch 4/10
374/374 [==============================] - 304s 812ms/step - loss: 1.3888 - acc: 0.2540 - val_loss: 1.3859 - val_acc: 0.2872


In [19]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [20]:
accr = model.evaluate(test_sequences_matrix,Y_test)

83/83 [==============================] - 1s 7ms/step


In [21]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 1.384
  Accuracy: 0.253
